In [68]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
from otgnn.models import GCN, compute_ot
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.mol_opt import MolOpt

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"
molopt = MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
)

In [70]:
embed = molopt.GCN(X)[0]
embed_tg = molopt.encode(X)

In [71]:
from mol_opt.transformer import *
model = make_model(args)
model

Encoder(
  (layers): ModuleList(
    (0): EncoderLayer(
      (self_attn): MultiHeadedAttention(
        (linears): ModuleList(
          (0): Linear(in_features=50, out_features=50, bias=True)
          (1): Linear(in_features=50, out_features=50, bias=True)
          (2): Linear(in_features=50, out_features=50, bias=True)
          (3): Linear(in_features=50, out_features=50, bias=True)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): PositionwiseFeedForward(
        (w_1): Linear(in_features=50, out_features=100, bias=True)
        (w_2): Linear(in_features=100, out_features=50, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (sublayer): ModuleList(
        (0): SublayerConnection(
          (norm): LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): SublayerConnection(
          (norm): LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (1): En

In [154]:
i = 0
X1 = embed.narrow(0, X.scope[i][0], X.scope[i][1])
X2 = embed_tg.narrow(0, i * args.dim_tangent_space, args.dim_tangent_space)

In [155]:
X2.shape

torch.Size([40, 50])

In [134]:
X1.shape

torch.Size([18, 50])

In [156]:
model(X1, None).shape

torch.Size([18, 50])

In [158]:
model(X2, None).shape

torch.Size([40, 50])

In [159]:
len(model.layers)

6

In [160]:
model.layers[0].forward(X1, None).shape

torch.Size([18, 50])

In [161]:
lr = model.layers[0]
lr.sublayer[0](X1, lambda x: lr.self_attn(x,x,x,None)).shape

torch.Size([18, 50])

In [162]:
lr.sublayer[0]

SublayerConnection(
  (norm): LayerNorm()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [163]:
f = lambda x: lr.self_attn(x,x,x,None)

In [164]:
f(X2).shape

torch.Size([40, 50])

In [165]:
X2.shape

torch.Size([40, 50])

In [166]:
attention(X2, X2, X2)[0].shape

torch.Size([40, 50])

In [167]:
X1.shape

torch.Size([18, 50])

In [108]:
lr.sublayer[0].norm().shape

torch.Size([18, 1, 50])

In [101]:
lr.self_attn(X1,X1,X1,None).shape

torch.Size([18, 1, 50])

In [78]:
model.layers[0].self_attn.forward(X1,X1,X1,None).shape

torch.Size([18, 1, 50])

In [80]:
X1.shape

torch.Size([18, 50])